In [1]:
# Install necessary packages

from Bio import SeqIO
from Bio.Data import CodonTable
import pandas as pd

# as a general note, I have a couple of lines commented-out at the end of each section that I used
    # to test my code

In [2]:
# 1.
# Dr. X's file calling function
# I changed the name of the function from "get_sequences_from_file' to "get_seq" because the original name was long
# I also changed the name of the dictionary from "sequence_data_dict" to "seqdat_dict"

def get_seq(fasta_fn):                                         # def defines the following as a function. Function given name. fasta_fn is the single argument of the function
    seqdat_dict = {}                                           # creates an empty dictionary
    for record in SeqIO.parse(fasta_fn, "fasta"):              # SeqIO reads SeqRecords. "fasta" tells it that the file in question is in FASTA format
                                                                   # SeqIO.parse separates the multiple FASTA records in one file
                                                                   # for loop loops through every record from the FASTA argument file parsed out and separated by SeqIO
        description = record.description.split()               # split the description metadata line that starts the FASTA record into a list of strings
        species_name = description[1] + " " + description[2]   # variable "species_name" is set to the 1st and 2nd items in "description" for the current record
        seqdat_dict[species_name] = record.seq                 # add the species name as the key and add the sequence of the current record to the dictionary
    return(seqdat_dict)                                        # output the contents of the dictionary
 
#print(get_seq("bears_cytb.fasta"))                             # call get_seq function with the bear cytochrome C fasta file as argument, importing sequence data

In [3]:
# 2. 
# Translation function utilizing codon table from FASTA file with multiple records
# Following outlines of pseudocode in sequence_translate.py

def translate_function(string_nucleotides):                                         # initiate and name function. Argument is a string of nucleotides                 
    mito_table = CodonTable.unambiguous_dna_by_name["Vertebrate Mitochondrial"]     # sets mito_table as the vertebrate mitochondrial codon table (a table showing which codons code for which amino acids)
                                                                                         # mitochondrial table used since cytochrome C is a mitochondrially produced protein - http://biopython.org/DIST/docs/tutorial/Tutorial.html
    aa_seq_string = []                                                              # create empty list   
    i = 3                                                                           # initialize i at 3
    for position in string_nucleotides:                                             # for loop through all bases in the sequence
        codon = str(string_nucleotides[(i-3):i:])                                   # set codon as the three bases. Specifically, the three bases between i and (i-3) are passed to codon
        if codon in {"AGA", "AGG", "TAA", "TAG"}:                                   # if codon is a mitochondrial stop codon - http://biopython.org/DIST/docs/tutorial/Tutorial.html, https://stackoverflow.com/questions/15112125/how-to-test-multiple-variables-against-a-value
            break                                                                        # break, leave for loop and don't translate stop codon - https://www.digitalocean.com/community/tutorials/how-to-use-break-continue-and-pass-statements-when-working-with-loops-in-python-3
        aa_seq_string += mito_table.forward_table[codon]                            # retrieve amino acid corresponding to the three bases in codon
        i += 3                                                                      # increment i by 3, moving to next codon in sequence
    aa_seq_string = ''.join(aa_seq_string)                                          # concatenate list of strings into string - https://stackoverflow.com/questions/12453580/concatenate-item-in-list-to-strings
    return(aa_seq_string)

#for key, value in (get_seq("bears_cytb.fasta")).items():                            # call get_seq to create dictionary with species name (key) and cytC sequence (value)
#    print(translate_function(value))                                                # call translate_function with the cytC sequence (value) as the argument and print

In [4]:
# 3.
# Translate using built-in Biopython capabilities - based on Dr. Friedburg's lecture "biopython_lecture-2018-11-07.pdf" on Slack

from Bio.Seq import Seq                                                             # import Seq object
from Bio.Alphabet import IUPAC                                                      # import IUPAC alphabets for Seq objects

def better_translate(seqobj_DNA):                                                   # define function - argument is a DNA string
    return(seqobj_DNA.translate(table = 2, to_stop = True))                         # use translate method to translate argument. Table 2 is the mitochondrial translation table. 
                                                                                        # setting "to_stop" equal to "True" causes stop codons to be ignored
#for key, value in (get_seq("bears_cytb.fasta")).items():                            # call get_seq to create dictionary with species name (key) and cytC sequence (value)
#    print(better_translate(value))

In [5]:
# 4.
# Molecular Weight function
# Following guidelines in sequence_translate.py

from Bio.SeqUtils.ProtParam import ProteinAnalysis                  # import necessary library

def mol_wt(string_aa):                                              # define function. Argument is an aa string
    seqobj_prot = ProteinAnalysis(string_aa)                        # ProteinAnalysis transforms the aa string into a protein Seq object - https://biopython.org/wiki/ProtParam
    return(seqobj_prot.molecular_weight())                          # the molecular_weight method calculates the molecular weight of the input protein Seq object

#for key, value in (get_seq("bears_cytb.fasta")).items():            # call get_seq to create dictionary with species name (key) and cytC sequence (value)
#    cytc_aa = str(better_translate(value))                          # turn output of get_seq into a string
#    print(mol_wt(cytc_aa))                                          

In [6]:
# 5.
# GC content function
# Didn't say if we were allowed to use Biopython, so I wrote my own

def GC_cont(string_nucleotides):                                     # define function. Argument is a string of nucleotides
    GC_count = 0                                                     # set GC count to 0
    for base in string_nucleotides:                                  # iterate through each base in the string of nucleotides
        if base in {"G", "C"}:                                       # if base is a G or a C
            GC_count += 1                                                # then increase the count by 1
    content = (GC_count/(len(string_nucleotides)))                   # GC content percentage is equal to the GC_count divided by the total length of the nucleotide sequence
    return(content)

#for key, value in (get_seq("bears_cytb.fasta")).items():            # call get_seq to create dictionary with species name (key) and cytC sequence (value)
#    print(GC_cont(value))

In [7]:
# 6. 
# Add new columns to dataframe

bears_mass = pd.read_csv("bears_mass.csv")                                     # import bears_mass.csv as dataframe
bears_copy = bears_mass.copy()                                                 # make copy of dataframe to keep original dataframe pristine
bears_copy = bears_copy.assign(molecular_weight = 'NaN', GC_content = 'NaN')   # add new columns using assign and populate them with 'NaN' - https://stackoverflow.com/questions/12555323/adding-new-column-to-existing-dataframe-in-python-pandas      
#bears_copy

In [8]:
# 7.
# Populate new columns with data using a combination of the functions made above

i = 0                                                               # set loop control variable to 0
j = 0                                                               # set loop control variable to 0

for key, value in (get_seq("bears_cytb.fasta")).items():            # call get_seq to create dictionary with species name (key) and cytC sequence (value)
    cytc_aa = str(better_translate(value))                          # turn output of get_seq into a string
    weight = (mol_wt(cytc_aa))                                      # call mol_wt to find the molecular weight of the translated cytV sequence
    GC = (GC_cont(value))                                           # call GC_cont to find the GC content of the translated cytC sequence
    
    for index, row in bears_copy.iterrows():                        # iterate through bears_copy dataframe
        bears_copy.loc[i, 'molecular_weight'] = weight              # replace the ith row of molecular_weight column with molecular weight of the current value (cytC sequence)
    i += 1                                                          # iterate i by 1
    
    for index, row in bears_copy.iterrows():                        # iterate through bears_copy dataframe
        bears_copy.loc[j, 'GC_content'] = GC                        # replace the jth row of GC_content column with GC content of the current value (cytC sequence)
    j += 1                                                          # iterate j by 1
        
bears_copy
    

,species,mass,molecular_weight,GC_content
0,Ursus spelaeus,550.800,42458.8,0.437719
1,Ursus arctos,203.500,42414.7,0.437719
2,Ursus thibetanus,99.714,42306.7,0.45614
3,Melursus ursinus,100.030,42552,0.451754
4,Ursus americanus,110.560,42427.7,0.439474
5,Helarctos malayanus,47.020,42560.9,0.442982
6,Ailuropoda melanoleuca,118.200,42702.2,0.407895
7,Tremarctos ornatus,140.700,42384.8,0.44386
8,Ursus maritimus,425.100,42454.8,0.442982
